In [1]:
# Import the packages we will use
import matplotlib.pyplot as plt
import numpy as np
import pandas
import h5py

import cooler

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# The following directive activates inline plotting
%matplotlib inline

In [3]:
# Download a high resoltion COOL file from the interwebs (this will take a few...)
# The ! at the beginning of the line tells IPython to run the line in the shell.
!wget http://makarich.fbb.msu.ru/agalicina/Lab_open/DICTY/COOL/Dicty_0A.1000.cool.multires

--2019-01-22 13:20:27--  http://makarich.fbb.msu.ru/agalicina/Lab_open/DICTY/COOL/Dicty_0A.1000.cool.multires
Распознаётся makarich.fbb.msu.ru (makarich.fbb.msu.ru)… 93.180.63.131
Подключение к makarich.fbb.msu.ru (makarich.fbb.msu.ru)|93.180.63.131|:80... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 78268942 (75M) [text/plain]
Сохранение в: «Dicty_0A.1000.cool.multires.1»

Dicty_0A.1000.cool. 100%[===================>]  74,64M  9,86MB/s    за 7,5s    

2019-01-22 13:20:35 (10,0 MB/s) - «Dicty_0A.1000.cool.multires.1» сохранён [78268942/78268942]



In [4]:
filepath = 'Dicty_0A.1000.cool.multires'

In [5]:
h5 = h5py.File(filepath, 'r')

In [6]:
h5

<HDF5 file "Dicty_0A.1000.cool.multires" (mode r)>

In [7]:
h5.keys()

KeysView(<HDF5 file "Dicty_0A.1000.cool.multires" (mode r)>)

In [8]:
h5['pixels']

KeyError: "Unable to open object (Object 'pixels' doesn't exist)"

In [ ]:
c = cooler.Cooler(filepath)